### 企业清单到基本表信息的名称，税号，注册类型和沪官界定日期列（B,C,D,E）

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
import os

path = os.getcwd()
# app = xw.App(visible=True,add_book=False)
wb = xw.Book(r'D:/egnail/file/企业信息及税收/企业清单（15-22年）.xlsx')
sht = wb.sheets['Sheet1']
col_d = sht.range('A:A').value
name = [[_] for _ in col_d[1:] if _ is not None]
LEN = len(name)
col_d = sht.range('C:C').value
n_tax = [[_] for _ in col_d[1:] if _ is not None]
col_d = sht.range('B:B').value[1: LEN+1]
type_register = [[_] for _ in col_d]
col_d = sht.range('E:E').value
date = [[_] for _ in col_d[1:] if _ is not None]
wb_basic = xw.Book(r'D:/egnail/file/企业信息及税收/泥城企业基本信息表.xlsx')
sht_tar = wb_basic.sheets['泥城所有企业']
sht_tar.range('B3').value = name
sht_tar.range('C3').value = n_tax
sht_tar.range('D3').value = type_register
sht_tar.range('E3').value = date
sht_tar.range('C3:C'+str(LEN+3)).api.NumberFormat = "0"
sht_tar.range('E3:E'+str(LEN+3)).api.NumberFormat = "yyyy-mm-dd"


NameError: name 'xw' is not defined

In [2]:
import pickle

my_nested_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# 保存嵌套列表到文件
with open('nested_list.pkl', 'wb') as file:
    pickle.dump(my_nested_list, file)

# 重新加载保存的嵌套列表对象
with open('nested_list.pkl', 'rb') as file:
    loaded_nested_list = pickle.load(file)


### 税收明细到基本表信息的2018年~2022年的年税收情况（L,M,N,O,P）

In [2]:
def find_position(rng, key):
    for i, con in enumerate(rng):
        for j, con_ in enumerate(con):
            if key == con_:
                output = [i, j]
                return output
home_path = r'D:\egnail\file\企业信息及税收\招商服务机构扶持明细（税收）'
f1, f2, f3, f4 = '四季度', '12', '税收合计', '税号'
f_doman = ['2018', '2019', '2020', '2021', '2022']
asc_start = 65
for _ in os.listdir(home_path):
    if (f1 in _) or (f2 in _):
        for _j in f_doman:
            if _j in _:
                year = _j
                wb_sub = xw.Book(home_path+'\\'+_)
                sht = wb_sub.sheets[0].range("A1:J4").value
                ps = find_position(sht, f3)
                col_lab = chr(asc_start+ps[1])
                row_lab = ps[0]
                ps = find_position(sht, f4)
                col_lab1 = chr(asc_start+ps[1])
                row_lab1 = ps[0]
                break
        tax_pyear = wb_sub.sheets[0].range(col_lab+':'+col_lab).value[row_lab+1: LEN+row_lab+1]
        tax_id = wb_sub.sheets[0].range(col_lab1+':'+col_lab1).value[row_lab1+1: LEN+row_lab1+1]
        tax_id_np = np.asarray(tax_id)
        n_tax_np = np.asarray(n_tax)
        row_id = []
        for _ in tax_id_np:
            row_id.append(np.where(n_tax_np==_))
        
        
        



TypeError: 'NoneType' object is not subscriptable

### Using pandas instead of xlwings module to analyze excel files.

In [6]:
import difflib
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import re
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string

def get_label_ind(sheet_head, col_indices):
    res = []
    for lab in col_indices:
        i = 0
        for _ in sheet_head:
            if lab in _.value:
                res.append(get_column_letter(i+1))
                break
            i += 1
    return res   

path = os.getcwd()
dest_file = r'/home/egnail/proj/NiCheng/企业信息及税收/泥城企业基本信息表.xlsx'
tax_folder = r'/home/egnail/proj/NiCheng/企业信息及税收/招商服务机构扶持明细（税收）/'
company_file = r'/home/egnail/proj/NiCheng/企业信息及税收/企业清单（15-22年）.xlsx'
wb_com = load_workbook(company_file)
sheet_com = wb_com.active
wb_dest = load_workbook(dest_file)
sheet_dest = wb_dest.active
col_indices1 = ['企业名称', '18位税号', '注册实体', '企业界定日期']
col_indices2 = ['企业名称', '税号', '注册型/实地型', '户管界定日期/注册时间']
col_ind_dest = get_label_ind(sheet_dest[2], col_indices2)

COLS = []
for i, _ in enumerate(col_indices1):
    for col in sheet_com.iter_cols(values_only=True):
        if _ == col[0]:
            COLS.append(col[1:])
            break
LEN = len(COLS[0])
COLS.insert(0, [i+1 for i in range(LEN)])
col_ind_dest.insert(0, 'A')
for j, _ in enumerate(COLS):
    for i, value in enumerate(_, start=3):
        cell = sheet_dest[col_ind_dest[j] + str(i)]
        cell.value = value
wb_dest.save(dest_file)

NAME_COMP = COLS[1]

### Collect tax data

In [7]:
def is_number(string):
    pattern = r'^-?\d+(\.\d+)?$'  # 匹配整数或小数形式的数字
    match = re.match(pattern, string)
    return match is not None
def search_company(name, ls):
    length = len(ls)
    similarity = np.zeros(length)
    for i, _ in enumerate(ls):
        similarity[i] = difflib.SequenceMatcher(autojunk=True, a=name, b=_).ratio()
    idx = np.argmax(similarity)
    return idx
ns = ['G', 'D' ,'F', 'D', 'E']  ##for 2021, 2019, 2018, 2022, 2020
col_indices3 = []
Name2, TAX = [], []
i = 0
header_row = sheet_dest[2]
for file in os.listdir(tax_folder):
    if '四季度' in file:
        abs_path = os.path.join(tax_folder, file)
        year = file.split('年')[0]
        col_indices3.append([_i.value for _i in header_row if year in _i.value][0])
        wb_tax = load_workbook(abs_path, data_only=True)
        sh_name = wb_tax.sheetnames[0]
        sheet_tax = wb_tax[sh_name]
        sub_name = []
        j = 0
        for col in sheet_tax.iter_cols(min_col=1, max_col=5, min_row=1, max_row=3, values_only=True):
            sub = tuple(filter(lambda x: x is not None, col))
            sub_name = ''.join(str(sub))
            if '企业名称' in sub_name:
                ns_ = get_column_letter(j+1)
                idd = col.index('企业名称')+1
                break
            j += 1
        for col in sheet_tax.iter_cols(min_col=column_index_from_string(ns_), max_col=column_index_from_string(ns_), values_only=True):
            Name2.append(col[idd:])
        for col in sheet_tax.iter_cols(min_col=column_index_from_string(ns[i]), max_col=column_index_from_string(ns[i]), values_only=True):
            TAX.append(col[idd:])
        i += 1
col_ind_dest = get_label_ind(header_row, col_indices3)
for j, _ in enumerate(TAX):
    for i, value in enumerate(_, start=3):
        val_name = Name2[j]
        idx = search_company(val_name[i], NAME_COMP)
        cell = sheet_dest[col_ind_dest[j] + str(idx)]
        cell.value = value

        

In [5]:
col.index('加补税合计')

1

In [64]:
for path, file_dir, files in os.walk(home_path):
    for file_name in files:
        # print(os.path.join(path, file_name))  # 当前循环打印的是当前目录下的所有文件
        print(file_name)
    # for dir in file_dir:
    #     print(os.path.join(path, dir)) 